In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
#%matplotlib inline 

import warnings
# both loading plastid and one of the lines below provoke some warnings
warnings.filterwarnings('ignore')

from plastid import *
from twobitreader import TwoBitFile
from collections import defaultdict
import six

#to make executable
import os

In [2]:
##All files are defined by their path rather than by a prefix.
os.system('pwd')

0

In [3]:
# what are the ensembl ids for these genes?
targeted_genes = pd.read_csv('target_gene_list.txt', header=None, names=['gene_name'])
feature_names = pd.read_csv('./references/features.tsv.gz',
                            sep='\t',
                            header=None,
                            names=['gene_id', 'gene_name', 'feature_type'])
targeted_genes = feature_names[feature_names['gene_name'].isin(targeted_genes['gene_name'])]

In [4]:
#check for genes that were not assigned an ensembl id
tmp = pd.read_csv('target_gene_list.txt', header=None, names=['gene_name'])
print('Missing:')
print(pd.DataFrame(tmp)[~pd.DataFrame(tmp)['gene_name'].isin(targeted_genes['gene_name'])])
#The gene names need to be updated for these genes. Search ensembl website to find desired gene name.
#Append these genes MANUALLY to the end of target_gene_list.txt and rerun this script.
#These genes must be added to target_gene_list.txt because this list is used by pick_targets.ipynb script.


Missing:
Empty DataFrame
Columns: [gene_name]
Index: []


In [5]:
# write these to a text document
np.savetxt('targeted_gene_ids.csv', targeted_genes['gene_id'].values, fmt='%s')

In [6]:
# # extract the relevant transcripts from the genome reference
os.system('grep -f targeted_gene_ids.csv /nvme/indices/refdata-cellranger-GRCh38-1.2.0/genes/genes.gtf > ./references/targeted_genes.gtf')


0

In [7]:
# # convert genome to 2bit format
os.system('faToTwoBit /nvme/indices/refdata-cellranger-GRCh38-1.2.0/fasta/genome.fa ./references/genome.2bit')


0

In [8]:
# this line produces a lot of warnings about duplicate tags...
annotated_transcripts = {transcript.attr['transcript_id']: transcript for transcript 
                             in GTF2_TranscriptAssembler('./references/targeted_genes.gtf',return_type=Transcript)}

In [9]:
# make a table of all the unique transcripts for each gene
targeted_transcripts = pd.Series({transcript_id: transcript.get_gene() for transcript_id, transcript in annotated_transcripts.iteritems()}).reset_index()
targeted_transcripts.columns = ['transcript_id', 'gene_id']

name_mapper = dict(zip(targeted_genes['gene_id'], targeted_genes['gene_name']))
id_mapper = dict(zip(targeted_genes['gene_name'], targeted_genes['gene_id']))

targeted_transcripts['gene_name'] = targeted_transcripts['gene_id'].map(name_mapper)
targeted_transcripts = targeted_transcripts.sort_values('gene_name').reset_index(drop=True)

In [10]:
# construct dictionary that groups transcript models by gene
transcripts_by_gene = defaultdict(dict)

for name, x in targeted_transcripts.iterrows():
    transcript_id = x['transcript_id']
    transcripts_by_gene[x['gene_name']][transcript_id] = annotated_transcripts[transcript_id]

In [11]:
# find spanning segments that cover all transcripts for a particular gene
spanning_segments = dict()

# we will add some extra space at the end just in case 3' UTRs aren't fully annotated
utr3_offset = 200

for gene, transcripts in transcripts_by_gene.iteritems():
    # note the spanning segments include annotated 3' UTRs
    start = np.min([x.spanning_segment.start for x in transcripts.values()])
    end = np.max([x.spanning_segment.end for x in transcripts.values()])
    
    sense = transcripts.values()[0].strand
    
    if sense == '+':
        end = end + utr3_offset
    elif sense == '-':
        start = start - utr3_offset
    
    spanning_segments[gene] = SegmentChain(GenomicSegment(transcripts.values()[0].chrom,
                                                          start, end,
                                                          transcripts.values()[0].strand),
                                          gene_id=id_mapper[gene])

In [12]:
import cPickle as pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [13]:
save_obj(spanning_segments, './counts/spanning_segments')

In [14]:
bam_list = {'CR2_l{0}'.format(i): '/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l{0}/outs/possorted_genome_bam.bam'.format(i) for i in range(1,7)}


In [15]:
for pretty_name, bam_file in bam_list.iteritems():
    alignments = BAMGenomeArray(bam_file)
    print(bam_file)
    print('================================================')
    
    # this will put weight 1/N on each nucleotide covered by a read, where N is the length of the read
    alignments.set_mapping(CenterMapFactory())

    # now accumulate read density
    count_vectors = dict()
    for gene, segment in spanning_segments.iteritems():
        print('\t{0}'.format(gene))
        count_vectors[gene] = segment.get_counts(alignments)
        
    save_obj(count_vectors, './counts/count_vectors_{0}.count'.format(pretty_name))

    # this will put weight 1 at the 5' end of the mapped read
    alignments.set_mapping(FivePrimeMapFactory())

    print('================================================')
    # now accumulate read density
    start_count_vectors = dict()
    for gene, segment in spanning_segments.iteritems():
        print('\t{0}'.format(gene))
        start_count_vectors[gene] = segment.get_counts(alignments)
        
    save_obj(start_count_vectors, './counts/start_count_vectors_{0}.count'.format(pretty_name))

/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l4/outs/possorted_genome_bam.bam
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l5/outs/possorted_genome_bam.bam
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l6/outs/possorted_genome_bam.bam
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l1/outs/possorted_genome_bam.bam
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l2/outs/possorted_genome_bam.bam
	TOMM7
	ASNA1
	TUBB
	ORC1
	FOXO3
	RPS5
	ATF4
	CDH3
	SEC61A1
	TOMM7
	ASNA1
	TUBB
	ORC1

In [16]:
# this is transcript-resolved mappings 5' end mappings, though with short reads many of these will end up randomly assigned
for pretty_name, bam_file in bam_list.iteritems():
    alignments = BAMGenomeArray(bam_file)
    alignments.set_mapping(FivePrimeMapFactory())
    
    print(bam_file)
    print('================================================')

    transcript_count_vectors = defaultdict(dict)

    for gene, transcripts in transcripts_by_gene.iteritems():
        print('\t{0}'.format(gene))
        for transcript_id, transcript in transcripts.iteritems():
            print('\t\t{0}'.format(transcript_id))
            transcript_count_vectors[gene][transcript_id] = transcript.get_counts(alignments)
            
    save_obj(transcript_count_vectors, './counts/transcript_count_vectors_{0}.count'.format(pretty_name))

/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l4/outs/possorted_genome_bam.bam
	TOMM7
		ENST00000483581
		ENST00000372879
		ENST00000463284
		ENST00000358435
		ENST00000405021
		ENST00000496129
		ENST00000621567
	ASNA1
		ENST00000590633
		ENST00000586561
		ENST00000611688
		ENST00000357332
		ENST00000591090
	TUBB
		ENST00000396389
		ENST00000327892
		ENST00000396384
		ENST00000330914
	ORC1
		ENST00000371566
		ENST00000371568
	FOXO3
		ENST00000406360
		ENST00000540898
		ENST00000343882
	RPS5
		ENST00000598807
		ENST00000598495
		ENST00000596046
		ENST00000196551
		ENST00000598098
		ENST00000596314
		ENST00000601521
		ENST00000599909
		ENST00000599232
	ATF4
		ENST00000396680
		ENST00000404241
		ENST00000337304
	CDH3
		ENST00000569080
		ENST00000569036
		ENST00000567674
		ENST00000542274
		ENST00000569117
		ENST00000429102
		ENST00000264012
		ENST00000568292
		ENST00000566808
		ENST00000565453
	SEC61A1
		ENST00000498837
		ENST00000491668
		ENST00000243253
		ENST00000481210
		ENST0

In [17]:
for pretty_name, bam_file in bam_list.iteritems():
    alignments = BAMGenomeArray(bam_file)
    alignments.set_mapping(CenterMapFactory())
    
    print(bam_file)
    print('================================================')

    transcript_count_vectors = defaultdict(dict)

    for gene, transcripts in transcripts_by_gene.iteritems():
        print('\t{0}'.format(gene))
        for transcript_id, transcript in transcripts.iteritems():
            print('\t\t{0}'.format(transcript_id))
            transcript_count_vectors[gene][transcript_id] = transcript.get_counts(alignments)
            
    save_obj(transcript_count_vectors, './counts/transcript_count_vectors_{0}.count'.format(pretty_name))

/home/jmr/direct_capture/doubles/perturbseq/count/CR2_l4/outs/possorted_genome_bam.bam
	TOMM7
		ENST00000483581
		ENST00000372879
		ENST00000463284
		ENST00000358435
		ENST00000405021
		ENST00000496129
		ENST00000621567
	ASNA1
		ENST00000590633
		ENST00000586561
		ENST00000611688
		ENST00000357332
		ENST00000591090
	TUBB
		ENST00000396389
		ENST00000327892
		ENST00000396384
		ENST00000330914
	ORC1
		ENST00000371566
		ENST00000371568
	FOXO3
		ENST00000406360
		ENST00000540898
		ENST00000343882
	RPS5
		ENST00000598807
		ENST00000598495
		ENST00000596046
		ENST00000196551
		ENST00000598098
		ENST00000596314
		ENST00000601521
		ENST00000599909
		ENST00000599232
	ATF4
		ENST00000396680
		ENST00000404241
		ENST00000337304
	CDH3
		ENST00000569080
		ENST00000569036
		ENST00000567674
		ENST00000542274
		ENST00000569117
		ENST00000429102
		ENST00000264012
		ENST00000568292
		ENST00000566808
		ENST00000565453
	SEC61A1
		ENST00000498837
		ENST00000491668
		ENST00000243253
		ENST00000481210
		ENST0

In [18]:
import glob

In [19]:
def get_files(pattern):
    file_list = glob.glob(pattern)
    all_count_vectors = dict()
    
    for file_name in file_list:
        all_count_vectors[file_name] = load_obj(file_name)
    return all_count_vectors

def merge_count_files(pattern):
    data = get_files(pattern)
    
    merged = dict()

    # initialize dictionary with values from first file
    first = data.pop(data.keys()[0])
    for gene_name, values in first.iteritems():
        merged[gene_name] = values.copy()

    # add on the rest
    for file, arrays in data.iteritems():
        for gene_name, values in arrays.iteritems():
            merged[gene_name] += values
            
    return merged

def merge_transcript_count_files(pattern):
    data = get_files(pattern)
    
    merged = defaultdict(dict)

    # initialize dictionary with values from first file
    first = data.pop(data.keys()[0])
    for gene_name, transcript_dict in first.iteritems():
        for transcript_id, values in transcript_dict.iteritems():
            merged[gene_name][transcript_id] = values.copy()

    # add on the rest
    for file, arrays in data.iteritems():
        for gene_name, transcript_dict in arrays.iteritems():
            for transcript_id, values in transcript_dict.iteritems():
                merged[gene_name][transcript_id] += values
            
    return merged

In [20]:
save_obj(merge_count_files('./counts/count_vectors*'), './counts/merged_count_vectors.count')

In [21]:
save_obj(merge_count_files('./counts/start_count_vectors*'), './counts/merged_start_count_vectors.count')

In [22]:
save_obj(merge_transcript_count_files('./counts/transcript_count_vectors*'), './counts/merged_transcript_count_vectors.count')